### Word2Vec - Skip Gram Model

Skip Gram model is implemented using tensorflow in this notebook.

For more information on this model, refer to following:
1. <a href = "https://arxiv.org/pdf/1301.3781.pdf">Efficient Estimation of Word Representations in Vector Space</a>
2. <a href = "https://arxiv.org/pdf/1310.4546.pdf">Distributed Representations of Words and Phrases and their Compositionality</a>

#### 1. Input Data

In [1]:
sentences = ['king is a strong man',
             'queen is a wise woman',
             'boy is a young man',
             'girl is a young woman',
             'prince is a young king',
             'princess is a young queen',
             'man is strong',
             'woman is pretty',
             'prince is a boy will be king',
             'princess is a girl will be queen'
            ]

#### 2. Getting rid of stopwords

In [2]:
def remove_stopwords(sentences):
    stopwords = {"is", "a", "will", "be"}
    
    transformed_sentences = []
    for idx, sentence in enumerate(sentences):
        result = list()
        for word in sentence.split(" "):
            if word in stopwords:
                continue
            result.append(word)
        transformed_sentences.append(" ".join(result))
    return transformed_sentences

In [3]:
sentences = remove_stopwords(sentences)

#### 3. Creating Vocabulary

In [4]:
vocabulary = set()
for sentence in sentences:
    for word in sentence.split(" "):
        vocabulary.add(word)

print("Size of the vocabulary: {}".format(len(vocabulary)))
print("Vocabulary: {}".format(vocabulary))

Size of the vocabulary: 12
Vocabulary: {'girl', 'woman', 'young', 'wise', 'king', 'boy', 'prince', 'man', 'queen', 'strong', 'princess', 'pretty'}
